In [ ]:
import datetime
import re
import requests
import model_financial
import model_tickers
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
url = 'https://github.com/E30895/smartFolio/raw/main/tickers_list.txt'
request = requests.get(url)

In [ ]:
def get_tickers() -> list:
    ''' Get many brazilians stock tickers from a text file and
        return a list.
    '''
    url = 'https://github.com/E30895/smartFolio/raw/main/tickers_list.txt'
    request = requests.get(url)
    tickers_list = request.text.split()
    request.close()

    return tickers_list

In [ ]:
selected_tickers = ['WEGE3.SA', 'PETR4.SA', 'MGLU3.SA']
start = '2015-06-13'
end = '2020-08-22'
interval = '1d'

In [ ]:
def main2(selected_tickers, start, end, interval):
    prices = model_financial.get_prices(selected_tickers, start, end, interval)
    #prices.set_index('Date', inplace=True)
    #returns = model_financial.log_return(prices)
    return prices

In [ ]:
prices = model_tickers.get_prices(selected_tickers, start=start, end=end, interval=interval)
returns = model_financial.log_return(prices)
wallets = model_financial.generate_wallets(returns.iloc[:, 1:])

In [33]:
def best_portfolio(wallets, method = 'sharpe_ratio'):
    vol = wallets['vol']
    sharpe = wallets['sharpe']
    weights = wallets['weights']
    returns = wallets['returns']
    
    if method == 'sharpe_ratio':

        indice = np.array(sharpe).argmax()

    elif method == 'volatility':

        indice = np.array(vol).argmin()

    elif method == 'return':

        indice = np.array(returns).argmax()

    return weights[indice]

In [ ]:
def plot_efficient_frontier(wallets, method = 'sharpe_ratio'):
    wallets = pd.DataFrame(wallets)
    vol = wallets['vol']
    returns = wallets['returns']
    sharpe = wallets['sharpe']
    

    if method == 'sharpe_ratio':

        indice = np.array(sharpe).argmax()
        y_axis = returns[indice]
        X_axis = vol[indice]

    elif method == 'volatility':

        indice = np.array(vol).argmin()
        y_axis = returns[indice]
        X_axis = vol[indice]

    elif method == 'return': 

        indice = np.array(returns).argmax()
        y_axis = returns[indice]
        X_axis = vol[indice]

    fig1 = px.scatter(x=wallets['vol'], y=wallets['returns'], color=wallets['sharpe'], color_continuous_scale='viridis',
                      labels={
                          'x':'Volatility',
                          'y':'Expected return',
                          'color':'Sharpe'
                      },
                      title = 'Efficient Frontier'
                      )
    fig1.update(layout_coloraxis_showscale=False)
    fig1.update_traces(showlegend=False, 
                       hovertemplate=''               
                       )

    #fig2 = px.scatter(x=[X_axis], y=[y_axis], color = 'red', size = 50)
    
    return fig1

In [ ]:
wallets = pd.DataFrame(wallets)

In [34]:
carteira_otima = best_portfolio(wallets)

In [37]:
carteira_otima

array([2.66525206e-01, 4.49697793e-04, 7.33025097e-01])

In [ ]:
fig1 = plot_efficient_frontier(wallets)

In [ ]:
indice = np.array(wallets['sharpe']).argmax()
y_axis = wallets['returns'][indice]
X_axis = wallets['vol'][indice]

In [32]:
fig = make_subplots(shared_yaxes=True, shared_xaxes=True)
fig.add_scatter(x=wallets['vol'], y=wallets['returns'], name='Carteiras')
fig.add_scatter(x=[X_axis], y=[y_axis], name='Carteira Ótima', marker_size = 10)
fig.update_traces(mode='markers', hovertemplate='Return: %{y}'+ '<br>Volatility: %{x}')
fig.update_layout(title='Efficient Frontier', xaxis_title='Volatility', yaxis_title='CPUs')